Summary:

- Transfer Learning: Load a pretrained model in pytorch and modify it to use it on other types of data
- Finetuning a model by freezing certain layers and only training a few layers


In [1]:
# 1. Imports
import torch
import torch.nn as nn # all neural network modules, nn.Linear, nn.Conv2d, BatchNorm, loss functions
import torch.optim as optim # all optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # all functions that dont have any parameters eg: activations like relu
from torch.utils.data import DataLoader # gives easier dataset management and creates mini batches
import torchvision.datasets as datasets # standard public datasets 
import torchvision.transforms as transforms # transforms on dataset
import torchvision.models as models # contains pretrained models and weights

In [2]:
# 4. Hyperparameters
image_size = 50
num_classes = 10
lr = 0.001
batch_size=64
num_epochs=10

In [3]:
# Load pretrained VGGNet and modify it
model = models.vgg16(pretrained=True)


/Users/kavyakumar/pytorch-notebooks/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kavyakumar/pytorch-notebooks/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
# for name, parameter in model.named_parameters():
#     print(name)
# print(model.features)
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [31]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x
'''
VGG16 is trained on images of shape (224,224,3) and num_classes = 1000
I want to modify it to train on CIFAR-10 dataset shape (512,512,3) and num_classes=10
I need to modify the linear layers of the classifier that get the
flattened depth channels from the last conv block and the avg pool layer dimensions
'''
X = torch.rand((64, 3, 512, 512))
model.avgpool = Identity() #replace avgpool layer. Now need to update linear layer input
model.classifier[0]=nn.Linear(model.avgpool.forward(X).shape[0]*model.avgpool.forward(X).shape[0], 50)
model.classifier[3]=nn.Linear(in_features=model.classifier[0].out_features,
                              out_features=model.classifier[0].out_features)
model.classifier[-1]=nn.Linear(in_features=model.classifier[3].out_features,
                              out_features=num_classes)
model.eval()
print(model(X).shape)



In [4]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
# Here in default training, we would perform backprop on all layers,
# which is unnecessary having leveraged pretrained weights. 
# One way is to finetune the model by freezing the weights up to 
# certain layers, and just running backprop on layers that require grad

In [11]:
import numpy as np

a = np.matrix([[1,2,3],[4,5,6]])
b = np.matrix([[7,8],[9,10],[11,12]])

print(a.shape, b.shape, a.dot(b), a.__matmul__(b))

(2, 3) (3, 2) [[ 58  64]
 [139 154]] [[ 58  64]
 [139 154]]
